In [1]:
using Knet, Plots, NBInclude;
nbinclude("deepppiutils.ipynb");

## Features Dictionary Construction

This function constructs protein features dictionary. Protein features for the taining data set are stored in the file "yeast_feature_all.csv". 
The function returns a dictionary mapping a protein UniProt ID to its 1164-features vector
* ```featurefilepath```    : The path for the file containint features data

In [2]:
function constructFeatDict(featurefilepath)
    # read features for all proteins
    f = open(featurefilepath);
    lines = readlines(f);
    close(f);
    numberOfProteins = length(lines) 
    featureNames = String.(split(lines[1],",")); 
    d = length(featureNames) - 1 # number of features per protein
    proteins = lines[2:numberOfProteins];
    featuresDict = Dict{String,Any}()
    for p in proteins
        featureVect = String.(split(p, ","));
        # to avoid NaN values due to missing data
        if("" in featureVect[2:d+1])
            continue
        end
        featuresDict[featureVect[1]] = parse.(Float32, featureVect[2:d+1]) # featureVect[1] is the protein UniProt ID 
    end
    return featuresDict;
end;

## Loading Data

This function loads the training data with its labels 
* ```featuresDict```    : The features dictionary which maps the protein ID to its features vector

The function returns two arrays, the data and the labels. 
In order to construct the data matrix, the following steps are executed:
* Protein pairs (represented by their UniProt IDs are read from the file "yeast_protein_pair.csv".
* A feature vector for each partner in the protein pair is extracted from ```featuresDict``` 
* The resulting two feature vectors are concatenated to form a new combined 2328-elements features vector
* Combined vectors for all pairs in the dataset are concatenated vertically to for the 65852x2328 data matrix ```concatAB```. 
* Labels for the protein pairs are read and returned as ```ygold```


In [3]:
function loaddata(featuresfilepath, pairsfilepath, nopos, noneg; Atype=gpu() >= 0 ? KnetArray{Float32} : Array{Float32})
    pairsfile = open(pairsfilepath)
    pairslines = readlines(pairsfile)
    featuresDict = constructFeatDict(featuresfilepath)
    close(pairsfile)
    
    n = length(pairslines); # number of samples/ protein pairs
    pairslines = pairslines[2:end];
    pos_a = pairslines[1:nopos];
    pos_b = pairslines[nopos+1:2*nopos];
    
    neg_a = pairslines[(2*nopos +1): (2*nopos+noneg)];
    neg_b = pairslines[(2*nopos+noneg+1):(2*nopos+2*noneg)];

    concatAB1 = []
    i = 1
    ygold = Array{UInt8,1}(nopos + noneg);

    pos_miss = 0
    for j in 1: length(pos_a)
        aline = pos_a[j]
        bline = pos_b[j]
        a = String.(split(aline, ","));
        b = String.(split(bline, ","));
        if(get(featuresDict, a[1], 0) == 0 || get(featuresDict, b[1], 0) == 0 )
            pos_miss += 1
            continue
        end
        push!(concatAB1, hcat(reshape(mat(featuresDict[a[1]]), 1, 1164), reshape(mat(featuresDict[b[1]]), 1, 1164)))
        ygold[i] = 0x02; # positive class
        i += 1
    end
    neg_miss = 0
    for j in 1:length(neg_a)
        aline = neg_a[j]
        bline = neg_b[j]
        a = String.(split(aline, ","));
        b = String.(split(bline, ","));
        if(get(featuresDict, a[1], 0) == 0 || get(featuresDict, b[1], 0) == 0 )
            neg_miss += 1
            continue
        end
        push!(concatAB1, hcat(reshape(mat(featuresDict[a[1]]), 1, 1164), reshape(mat(featuresDict[b[1]]), 1, 1164)))
        ygold[i] = 0x01; # negative class
        i += 1
    end
    println("Num of dropped positive sample: ", pos_miss, " , Num of dropped negative sample: ", neg_miss)
    concatAB = vcat(map(Atype, concatAB1)...)
    return concatAB, ygold
end;

In [4]:
function trn_tst_split(data, ygold, trnper, tstper; batchsize=64)
    nosamples = size(data,1)
    notrn = Int(floor(trnper * nosamples))
    notst = nosamples - notrn
    ind = randperm(nosamples)
    
    xtrn = data[ind[1:notrn],:];
    ytrn = ygold[ind[1:notrn]];
    
    xtst = data[ind[notrn+1:notrn+notst], :];
    ytst = ygold[ind[notrn+1:notrn+notst]];
    
    dtrn = minibatchi(xtrn',ytrn,batchsize);
    dtst = minibatchi(xtst',ytst,batchsize);
    
    return dtrn, dtst
end;

In [5]:
w_sizes = Tuple{Int64,Int64}[(512, 1164) (512, 1) (256, 512) (256, 1) (128, 256) (128, 1) (512, 1164) (512, 1) (256, 512) (256, 1) (128, 256) (128, 1) (128, 256) (128, 1) (2, 128) (2, 1)]
wl = loadmodel("DeepPPI_SepModel5.csv",  w_sizes);

# Yeast Dataset

In [7]:
summary(yeast_dataset)

"11188×2328 Array{Float32,2}"

In [6]:
yeast_feature="test_datasets/Yeast/yeast_feature.csv"
yeast_protein="test_datasets/Yeast/yeast_protein.csv"
yeast_dataset,yeast_label=loaddata(yeast_feature,yeast_protein,5594,5594);
#dtrn, dtst = trn_tst_split(yeast_dataset,yeast_label, 0.75, 0.25);

Num of dropped positive sample: 0 , Num of dropped negative sample: 0


In [8]:
# the number of hidden units in the hidden layers of the DeepPPI-CON model
HIDDENSSEP = Any[NOINPUTS, 512, 256, 128]; 
HIDDENSMER = Any[256, 128, NOOUTPUTS]
NOEPOCH = 30;
BATCHSIZE = 64;
PDROP = (0, 0.2);
accuracy_ =[]
recall=[]
specifity=[]
precision= []
mcc=[]
f1=[]
npv=[]
accuracyt= recalli=specifityi=precisioni=mcci = 0.0
for i in 1:5
    #setseed(i);
    #wa = winit(HIDDENSSEP...);
    #wb = winit(HIDDENSSEP...);
    #wMerged = winit(HIDDENSMER...);
    #w = vcat(wa, wb, wMerged);
    w = deepcopy(wl)
    
    #dtrn, ddev, dtst = dividedataset(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE);
    dtrn, dtst = trn_tst_split(yeast_dataset, yeast_label, trnper, tstper; batchsize= BATCHSIZE);
    
    optims = params(w; optim="Momentum", lr=0.01, gamma=0.9);
    #@time trnloss, trnerr, tstloss, tsterr=trainSep!(w, optims, dtrn, predictSep, ddev; pdrop=PDROP, epochs=NOEPOCH) 
    @time trainSep!(w, optims, dtrn, predictSep, dtst; pdrop=PDROP, epochs=NOEPOCH) 
    
    println("Yeast Dataset, fold: ", i)
    println("Training: min. loss =",loss(w,dtrn,predictSep),", min. error =",zeroone(w,dtrn,predictSep))  
    println("Test: min. loss =",loss(w,dtst,predictSep),", min. error =",zeroone(w,dtst,predictSep))  
    
    accuracyt,recalli,specifityi,precisioni,mcci,f1i,npvi = modelevaluation(w, dtst, predictSep; p=true);
    push!(accuracy_, accuracyt)
    push!(recall, recalli)
    push!(specifity, specifityi)
    push!(precision, precisioni)
    push!(mcc, mcci)
    push!(f1, f1i)
    push!(npv, npvi)
    
end

(:epoch, 10, :trn, 0.9692234848484849, :tst, 0.931573547979798)
(:epoch, 20, :trn, 0.9907670454545454, :tst, 0.9416666666666665)
(:epoch, 30, :trn, 0.9959753787878788, :tst, 0.9375552398989899)
108.148394 seconds (9.39 M allocations: 39.459 GiB, 4.80% gc time)
Yeast Dataset, fold: 1
Training: min. loss =0.013052665, min. error =0.0040246212121212155
Test: min. loss =0.3003736, min. error =0.06244476010101008
TP: 1234 , TN: 1388 , FP: 112 , FN: 63
Model evaluation:
Accuracy : 0.9374329635547255
Precision : 0.916790489660631
NPV : 0.9565816671560429
Sensitivity / Recall : 0.9514263678092317
Specifity : 0.9253333327164444
MCC : 0.8750642907877682
F1 : 0.9337873624919458
(:epoch, 10, :trn, 0.9635416666666666, :tst, 0.9377604166666665)
(:epoch, 20, :trn, 0.9924242424242424, :tst, 0.9429371843434343)
(:epoch, 30, :trn, 0.9952651515151515, :tst, 0.9479087752525253)
 87.800304 seconds (7.55 M allocations: 39.367 GiB, 5.00% gc time)
Yeast Dataset, fold: 2
Training: min. loss =0.01340518, min. e

In [9]:
open("DeepPPI-YeastDataset_Scores.txt", "w") do f
    write(f, "Dataset \tAccuracy\t\t\tPrecision\t\t\tnpv      \t\t\tRecall   \t\t\tSpecifity\t\t\tMCC\n")
    write(f, "__________________________________________________________________________________________________________________________________\n")
    for i in 1:5
        write(f, "dataset"*string(i)*"\t"*string(accuracy_[i]) *"\t"* string(precision[i]) *"\t"* string(npv[i]) *"\t"* string(recall[i]) *"\t"*  string(specifity[i]) *"\t"*  string(mcc[i]) *"\n")
    end
    write(f, "__________________________________________________________________________________________________________________________________\n")
    write(f, "Average"*"\t\t"*string(mean(accuracy_)) *"\t"* string(mean(precision))  *"\t"* string(mean(npv)) *"\t"* string(mean(recall)) *"\t"*  string(mean(specifity)) *"\t"*  string(mean(mcc)) *"\n")
end;
## DeepPPI: acc=0.948159, precision=0.972388, npv=0.925875, sensitivity=0.923459, specificity=0.973304, mcc=0.897513

# Pylori Dataset

In [10]:
pylori_feature="test_datasets/pylori/pylori_feature.csv"
pylori_protein="test_datasets/pylori/pylori_protein.csv"
pylori_dataset,pylori_label=loaddata(pylori_feature,pylori_protein,1458,1458);
#dtrn, dtst = trn_tst_split(pylori_dataset,pylori_label, 0.75, 0.25);

Num of dropped positive sample: 0 , Num of dropped negative sample: 34


In [11]:
# the number of hidden units in the hidden layers of the DeepPPI-CON model
HIDDENSSEP = Any[NOINPUTS, 512, 256, 128]; 
HIDDENSMER = Any[256, 128, NOOUTPUTS]
NOEPOCH = 30;
BATCHSIZE = 64;
PDROP = (0, 0.2);

accuracy_ =[]
recall=[]
specifity=[]
precision= []
mcc=[]
f1=[]
npv=[]
accuracyt= recalli=specifityi=precisioni=mcci = 0.0
for i in 1:5
    #setseed(i);
    #wa = winit(HIDDENSSEP...);
    #wb = winit(HIDDENSSEP...);
    #wMerged = winit(HIDDENSMER...);
    #w = vcat(wa, wb, wMerged);
    w = deepcopy(wl)
    #dtrn, ddev, dtst = dividedataset(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE);
    dtrn, dtst = trn_tst_split(pylori_dataset,pylori_label, trnper, tstper; batchsize= BATCHSIZE);
    
    optims = params(w; optim="Momentum", lr=0.01, gamma=0.9);
    #@time trnloss, trnerr, tstloss, tsterr=trainSep!(w, optims, dtrn, predictSep, ddev; pdrop=PDROP, epochs=NOEPOCH) 
    @time trainSep!(w, optims, dtrn, predictSep, dtst; pdrop=PDROP, epochs=NOEPOCH) 
    
    println("Yeast Dataset, fold: ", i)
    println("Training: min. loss =",loss(w,dtrn,predictSep),", min. error =",zeroone(w,dtrn,predictSep))  
    println("Test: min. loss =",loss(w,dtst,predictSep),", min. error =",zeroone(w,dtst,predictSep))  
    
    accuracyt,recalli,specifityi,precisioni,mcci,f1i,npvi = modelevaluation(w, dtst, predictSep; p=true);
    push!(accuracy_, accuracyt)
    push!(recall, recalli)
    push!(specifity, specifityi)
    push!(precision, precisioni)
    push!(mcc, mcci)
    push!(f1, f1i)
    push!(npv, npvi)
    
end

(:epoch, 10, :trn, 0.9624943727490997, :tst, 0.8622855392156863)
(:epoch, 20, :trn, 0.9912683823529411, :tst, 0.8531709558823529)
(:epoch, 30, :trn, 0.9963235294117647, :tst, 0.8544730392156863)
 21.219558 seconds (1.95 M allocations: 10.144 GiB, 5.58% gc time)
Yeast Dataset, fold: 1
Training: min. loss =0.0155166285, min. error =0.003676470588235281
Test: min. loss =0.7072286, min. error =0.1455269607843137
TP: 315 , TN: 297 , FP: 51 , FN: 58
Model evaluation:
Accuracy : 0.8488210806535075
Precision : 0.8606557353533997
NPV : 0.8366197159531839
Sensitivity / Recall : 0.8445040191836354
Specifity : 0.8534482734096314
MCC : 0.6976137945710008
F1 : 0.8525033817963419
(:epoch, 10, :trn, 0.9670899609843938, :tst, 0.843060661764706)
(:epoch, 20, :trn, 0.9972426470588235, :tst, 0.8323376225490197)
(:epoch, 30, :trn, 0.9990808823529411, :tst, 0.8378523284313726)
 21.355732 seconds (1.95 M allocations: 10.144 GiB, 5.81% gc time)
Yeast Dataset, fold: 2
Training: min. loss =0.005822806, min. err

In [12]:
open("DeepPPI-PyloriDataset_Scores.txt", "w") do f
    write(f, "Dataset \tAccuracy\t\t\tPrecision\t\t\tnpv      \t\t\tRecall   \t\t\tSpecifity\t\t\tMCC\n")
    write(f, "__________________________________________________________________________________________________________________________________\n")
    for i in 1:5
        write(f, "dataset"*string(i)*"\t"*string(accuracy_[i]) *"\t"* string(precision[i]) *"\t"* string(npv[i]) *"\t"* string(recall[i]) *"\t"*  string(specifity[i]) *"\t"*  string(mcc[i]) *"\n")
    end
    write(f, "__________________________________________________________________________________________________________________________________\n")
    write(f, "Average"*"\t\t"*string(mean(accuracy_)) *"\t"* string(mean(precision))  *"\t"* string(mean(npv)) *"\t"* string(mean(recall)) *"\t"*  string(mean(specifity)) *"\t"*  string(mean(mcc)) *"\n")
end;

# Human Dataset

In [13]:
human_feature="test_datasets/Human/human_feature.csv"
human_protein="test_datasets/Human/human_protein.csv"
human_dataset,human_label=loaddata(human_feature,human_protein,3899,4262);

Num of dropped positive sample: 0 , Num of dropped negative sample: 0


In [14]:
# the number of hidden units in the hidden layers of the DeepPPI-CON model
HIDDENSSEP = Any[NOINPUTS, 512, 256, 128]; 
HIDDENSMER = Any[256, 128, NOOUTPUTS]
NOEPOCH = 30;
BATCHSIZE = 64;
PDROP = (0, 0.2);

accuracy_ =[]
recall=[]
specifity=[]
precision= []
mcc=[]
f1=[]
npv=[]
accuracyt= recalli=specifityi=precisioni=mcci = 0.0
for i in 1:5
    #setseed(i);
    #wa = winit(HIDDENSSEP...);
    #wb = winit(HIDDENSSEP...);
    #wMerged = winit(HIDDENSMER...);
    #w = vcat(wa, wb, wMerged);
    w = deepcopy(wl)
    #dtrn, ddev, dtst = dividedataset(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE);
    dtrn, dtst = trn_tst_split(human_dataset,human_label, trnper, tstper; batchsize= BATCHSIZE);
    
    optims = params(w; optim="Momentum", lr=0.01, gamma=0.9);
    #@time trnloss, trnerr, tstloss, tsterr=trainSep!(w, optims, dtrn, predictSep, ddev; pdrop=PDROP, epochs=NOEPOCH) 
    @time trainSep!(w, optims, dtrn, predictSep, dtst; pdrop=PDROP, epochs=NOEPOCH) 
    
    println("Human Dataset, fold: ", i)
    println("Training: min. loss =",loss(w,dtrn,predictSep),", min. error =",zeroone(w,dtrn,predictSep))  
    println("Test: min. loss =",loss(w,dtst,predictSep),", min. error =",zeroone(w,dtst,predictSep))  
    
    accuracyt,recalli,specifityi,precisioni,mcci,f1i,npvi = modelevaluation(w, dtst, predictSep; p=true);
    push!(accuracy_, accuracyt)
    push!(recall, recalli)
    push!(specifity, specifityi)
    push!(precision, precisioni)
    push!(mcc, mcci)
    push!(f1, f1i)
    push!(npv, npvi)
    
end

(:epoch, 10, :trn, 0.9951822916666666, :tst, 0.9706431606359649)
(:epoch, 20, :trn, 0.9986979166666666, :tst, 0.9745494106359649)
(:epoch, 30, :trn, 0.9998372395833334, :tst, 0.97265625)
 59.254573 seconds (5.49 M allocations: 28.666 GiB, 5.21% gc time)
Human Dataset, fold: 1
Training: min. loss =0.0023086087, min. error =0.00016276041666662966
Test: min. loss =0.15141505, min. error =0.02734375
TP: 923 , TN: 1062 , FP: 51 , FN: 5
Model evaluation:
Accuracy : 0.9725624689012432
Precision : 0.9476386027231637
NPV : 0.9953139634533139
Sensitivity / Recall : 0.994612067893737
Specifity : 0.9541778967168212
MCC : 0.9458667641467686
F1 : 0.9705573075864578
(:epoch, 10, :trn, 0.99755859375, :tst, 0.9779673793859649)
(:epoch, 20, :trn, 0.9954427083333334, :tst, 0.9613658168859649)
(:epoch, 30, :trn, 0.9996744791666666, :tst, 0.9755259731359649)
 59.820012 seconds (5.49 M allocations: 28.666 GiB, 5.17% gc time)
Human Dataset, fold: 2
Training: min. loss =0.0011807407, min. error =0.00032552083

In [15]:
open("DeepPPI-HumanDataset_Scores.txt", "w") do f
    write(f, "Dataset \tAccuracy\t\t\tPrecision\t\t\tnpv      \t\t\tRecall   \t\t\tSpecifity\t\t\tMCC\n")
    write(f, "__________________________________________________________________________________________________________________________________\n")
    for i in 1:5
        write(f, "dataset"*string(i)*"\t"*string(accuracy_[i]) *"\t"* string(precision[i]) *"\t"* string(npv[i]) *"\t"* string(recall[i]) *"\t"*  string(specifity[i]) *"\t"*  string(mcc[i]) *"\n")
    end
    write(f, "__________________________________________________________________________________________________________________________________\n")
    write(f, "Average"*"\t\t"*string(mean(accuracy_)) *"\t"* string(mean(precision))  *"\t"* string(mean(npv)) *"\t"* string(mean(recall)) *"\t"*  string(mean(specifity)) *"\t"*  string(mean(mcc)) *"\n")
end;

## Base model

In [19]:
dtrnyeast, dtstyeast = trn_tst_split(yeast_dataset, yeast_label, 1.0, 0; batchsize= 64);
dtrnhuman, dtsthuman = trn_tst_split(human_dataset,human_label, 1.0, 0; batchsize= 64);
dtrnpylori, dtstpylori = trn_tst_split(pylori_dataset,pylori_label, 1.0, 0; batchsize= 64);

In [20]:
# Training Set
trnacc = 0;
for (x, y) in dtrnyeast
    ypred = predictSep(wl,x)
    trnacc += accuracyi(ypred, y) 
end
println(trnacc/length(dtrnyeast));
println(loss(wl, dtrnyeast, predictSep));
modelevaluation(wl, dtrnyeast, predictSep; p=true);

0.44370879120879125
5.358826
TP: 3845 , TN: 1119 , FP: 1749 , FN: 4475
Model evaluation:
Accuracy : 0.44368966746123617
Precision : 0.6873435822868531
NPV : 0.20003575255630393
Sensitivity / Recall : 0.46213942302137745
Specifity : 0.3901673638806948
MCC : -0.12897018203148128
F1 : 0.5526807531584964


In [21]:
# Training Set
trnacc = 0;
for (x, y) in dtrnhuman
    ypred = predictSep(wl,x)
    trnacc += accuracyi(ypred, y) 
end
println(trnacc/length(dtrnhuman));
println(loss(wl, dtrnhuman, predictSep));
modelevaluation(wl, dtrnhuman, predictSep; p=true);

0.5007583155776515
4.3179364
TP: 2892 , TN: 1198 , FP: 1007 , FN: 3064
Model evaluation:
Accuracy : 0.5011640729688562
Precision : 0.7417286481811417
NPV : 0.28108869068956155
Sensitivity / Recall : 0.48556077896481514
Specifity : 0.5433106573499725
MCC : 0.025666502702106
F1 : 0.5869101978095475


In [22]:
# Training Set
trnacc = 0;
for (x, y) in dtrnpylori
    ypred = predictSep(wl,x)
    trnacc += accuracyi(ypred, y) 
end
println(trnacc/length(dtrnpylori));
println(loss(wl, dtrnpylori, predictSep));
modelevaluation(wl, dtrnpylori, predictSep; p=true);

0.4670516304347826
4.8245883
TP: 976 , TN: 368 , FP: 482 , FN: 1056
Model evaluation:
Accuracy : 0.46634281732604344
Precision : 0.6694101504325033
NPV : 0.2584269661106552
Sensitivity / Recall : 0.48031496039354576
Specifity : 0.43294117596124565
MCC : -0.079118185099487
F1 : 0.559312320756644
